In [5]:

import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.tools import tool
from typing import TypedDict, List
from langchain_core.messages import BaseMessage
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage


load_dotenv()
API_KEY=os.getenv("ARK_API_KEY")
MODEL=os.getenv("MODEL")
BASE_URL=os.getenv("BASE_URL")
# 定义工具
@tool
def save_user_name(name: str) -> str:
    """保存用户名

    Args:
        name: 用户的名字
    """
    return f"已记住你的名字：{name}"

@tool
def get_user_name() -> str:
    """获取用户名"""
    # 注意：这个简化版本无法真正记忆
    # 真正的记忆需要在状态中维护
    return "请先告诉我你的名字"

# 创建模型和 Agent
model=init_chat_model(
    model=MODEL,
    model_provider="openai",
    api_key=API_KEY,
    base_url=BASE_URL,
    temperature=0.3
)

agent = create_agent(
    model=model,
    tools=[save_user_name, get_user_name],
    system_prompt="""你是一个记忆助手。

当用户告诉你名字时，使用 save_user_name 工具保存。
当用户询问名字时，使用 get_user_name 工具查询。

重要：你要记住对话历史中的信息，不要每次都重新询问。"""
)

# 测试
print("🤖 记忆助手测试\n")

messages = []

# 轮次1：告知名字
print("【轮次1】")
user_input = "我叫张三"
print(f"👤 用户: {user_input}")

messages.append({"role": "user", "content": user_input})
response=""
for chunk in model.stream(messages):
    print(chunk.content,end="",flush=True)
    response+=chunk.content
messages.append(AIMessage(content=response))

# result = agent.invoke({"messages": messages})
# messages = result["messages"]

# ai_response = messages[-1].content
# print(f"🤖 AI: {ai_response}\n")

# 轮次2：询问名字
print("【轮次2】")
user_input = "我叫什么名字？"
print(f"👤 用户: {user_input}")

messages.append({"role": "user", "content": user_input})
# result = agent.invoke({"messages": messages})
# messages = result["messages"]

# ai_response = messages[-1].content
# print(f"🤖 AI: {ai_response}\n")
response=""
for chunk in model.stream(messages):
    print(chunk.content,end="",flush=True)
    response+=chunk.content
messages.append(AIMessage(content=response))
     

🤖 记忆助手测试

【轮次1】
👤 用户: 我叫张三
你好呀，张三！很高兴认识你～ 有什么想聊的或者需要帮忙的，随时告诉我哦～ 😊【轮次2】
👤 用户: 我叫什么名字？
你叫张三呀～ 我记得很清楚呢！😊 还有什么想聊的吗？